In [2]:
import urllib.request
from bs4 import BeautifulSoup

soup = BeautifulSoup(urllib.request.urlopen("https://www.goszakup.gov.kz/ru/registry/rqc").read(),'lxml')

tbody = soup('table', {"class": "table table-bordered table-hover"})[0].find_all('tr')

for row in tbody:
    cols = row.findChildren(recursive=False)
    cols = [ele.text.strip() for ele in cols]
    print(cols)
    
#В сайте были убраны данные :(

['№', 'Наименование потенциального поставщика', 'БИН/ИИН', 'Наименование, номер и дата выдачи документа, на основании которого потенциальный поставщик включен в Перечень']


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [4]:
url = 'https://www.goszakup.gov.kz/ru/registry/rqc'
driver = webdriver.Chrome() 
driver.get(url)

In [5]:
from selenium.webdriver.chrome.options import Options

def extracting_name_iin(link):
    
    soup = BeautifulSoup(urllib.request.urlopen(link).read(),'lxml')

    table = soup.find_all('table')
    if not table:
        return pd.DataFrame({'Column1': [np.nan], 'Column2': [np.nan], 'Address': [np.nan]})

    df1 = pd.read_html(str(table))[2].drop([1,1]).T.drop([0,0]).reset_index()
    df2 = pd.read_html(str(table))[3]
    extracted_col = df2["Полный адрес(рус)"]
    
    extracted_col = df2["Полный адрес(рус)"].str.cat(sep='; ')

    new_dataframe = pd.concat([df1, pd.Series([extracted_col], name="Address")], axis=1).drop(['index'], axis=1)
    return new_dataframe


soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup('table')[0]
final_df = pd.read_html(str(table))[0]

dfs = []
for row in table.find_all('tr'):
    for link in row.find_all('a', href=True):
        df = extracting_name_iin(link['href'])
        dfs.append(df)
        display(df)

        
big_dataframe = pd.concat(dfs, ignore_index = True)

result_df = pd.concat([final_df, big_dataframe], axis=1) #result

result_df = result_df.drop_duplicates(subset=['Наименование потенциального поставщика'])

result_df.reset_index(drop=True, inplace=True)

result_df.to_excel('result.xlsx', index=False)


,№,Наименование потенциального поставщика,БИН/ИИН,"Наименование, номер и дата выдачи документа, на основании которого потенциальный поставщик включен в Перечень"
